In [32]:
import pydicom
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
from skimage.transform import resize
import os
import SimpleITK as sitk
from array import array

In [2]:
data_path = '/home/g0/Projects/singlechip-pct/GateContrib/imaging/CT/fixedForcedDetectionCT/data/'
file_name = 'ICRP_AF.mhd'
example_path = '/home/g0/Projects/singlechip-pct/phantom/mhd/example_data/'
examples = ['chest_ct.mhd', 'chest_xray.mhd']

In [3]:
im_size = (512, 512)

In [4]:

phantom_data = {
    "catphan600": {
        "edge": {
            "R": 0,
            "r": 100, "HU": 1000
        },
        "main": {
            "R": 0,
            "r": 75, "HU": 1100
        },
        "sen":{
            "R": 58, "r": 6.5,
            "angles": [np.deg2rad(i) for i in [0, 30, 90, 150, 180, 240, 270, 330]],
            "HU": [0, 1950, 1350, 1135, 0, 965, 915, 820],
            "M": ["air", "Teflon", "Delrin", "Acrylic", "vial-H2O", "Polystylrene", "LDPE", "PMP"]
        },
        "pix": {
            "R": 25*np.sqrt(2), "r": 1.5,
            "angles": [np.deg2rad(i) for i in [45, 135, 225, 315]],
            "HU": [1950, 0, 0, 0],
            "M": ["Teflon"] + 3*["hole"]
        },
        "cen": {
            "R": 15, "r": [5, 1, 2, 3, 4],
            "angles": [np.deg2rad(i*360/5) for i in range(5)],
            "HU": 1135
        }
    }
}


In [5]:

def draw_img(arr_img, phantom, pspace):
    for key, value in phantom.items():
        if key in ["edge", "main"]:
            r = value["r"]/pspace
            for row in range(len(arr_img)):
                for col in range(len(arr_img[0])):
                    if (row - len(arr_img)/2)**2 + (col - len(arr_img[0])/2)**2 <= r**2:
                        arr_img[row][col] = value["HU"]
        else :
            for m in range(len(value["angles"])):
                r = 0.0
                if key == "cen":
                    r = value["r"][m]/pspace
                    HU = value["HU"]
                else:
                    HU = value["HU"][m]
                    r = value["r"]/pspace
                
                R = value["R"]/pspace
                for row in range(len(arr_img)):
                    for col in range(len(arr_img[0])):
                        if (row + R*np.cos(value["angles"][m]) - len(arr_img)/2)**2 +\
                            (col - R*np.sin(value["angles"][m]) - len(arr_img[0])/2)**2 <=\
                                r**2:
                            arr_img[row][col] = HU

In [6]:
def get_fdcm(file_path):
    ds = pydicom.dcmread(file_path)
    return ds
ds = get_fdcm('/home/g0/Projects/singlechip-pct/image/dicom/catphan600/xct/CT.1.2.840.113704.1.111.4484.1506491899.1946.dcm')

In [7]:
arr_img = np.zeros(im_size)
arr_img_hq = np.zeros((im_size[0]*4, im_size[0]*4))
draw_img(arr_img, phantom_data["catphan600"], ds.PixelSpacing[0])
draw_img(arr_img_hq, phantom_data["catphan600"], ds.PixelSpacing[0]/4)

In [8]:
arr_img_rehq = resize(arr_img_hq, (512, 512), preserve_range=True, anti_aliasing=True)

In [9]:
# plt.figure(figsize=(15, 15))
# plt.imshow(arr_img_rehq, cmap='gray')

In [52]:
def arrimg2_3d(arr, h=0):
    if h:
        return np.array([arr.copy() for i in range(h)])
    else:
        return arr.copy()
    
def get_imgitk(arr):
    return sitk.GetImageFromArray(arr)

def save_bimg(arr_img, fname):
    fpath = "./output/"
    f = open(fpath + fname + '.png', 'wb')
    byte_array = bytearray(arr_img)
    f.write(byte_array)
    f.close()
    
def read_bim(fpath):
    input_file = open(fpath, 'rb')
    array = input_file.read()
    array = np.frombuffer(array)
    return array

In [43]:
save_bimg(arr_img, "arr_img")
save_bimg(arr_img_hq, "arr_img_hq")
save_bimg(arr_img_rehq, "arr_img_rehq")
# itk_img = get_imgitk(arr_img_hq)
# arr_img_hq_3d = arrimg2_3d(arr_img_hq, 1024)
# itk_img2 = get_imgitk(arr_img_hq_3d)

In [87]:
arr = read_bim("/home/g0/Projects/singlechip-pct/phantom/mhd/output/arr_img_rehq.png")
print(arr.shape)
arr_2 = np.reshape(arr, (-1, 512))
arr_img_3d = arrimg2_3d(arr_2, 128)

(262144,)


In [80]:
# plt.figure(figsize = (15, 15))
# plt.imshow(new_itk_img_array[100], cmap='gray')

In [94]:
itk_img = get_imgitk(arr_img_3d)
itk_img.SetSpacing(3*[ds.PixelSpacing[0]])
sitk.WriteImage(itk_img, 'output/mhd/catphan404/img.mhd')


In [89]:
new_itk_img_array = sitk.GetImageFromArray([arr_img_3d[:, :, i] for i in range(len(arr_img_3d[0, 0, :]))])

In [90]:
sitk.Show(new_itk_img_array, 'sample image')

OpenJDK 64-Bit Server VM warning: ignoring option PermSize=128m; support was removed in 8.0
OpenJDK 64-Bit Server VM warning: Using incremental CMS is deprecated and will likely be removed in a future release


[INFO] Detected existing ImageJ; passing arguments along
